In [15]:
!pip install tf2onnx
!pip install tensorflow_ranking

In [16]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Reshape, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.initializers import GlorotUniform
import tensorflow as tf
import tensorflow_ranking as tfr
import tf2onnx
from keras.utils.np_utils import to_categorical

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
input_path = 'drive/MyDrive/CURIS/VeriX/train_networks/gtsrb.pickle'
output_path = 'drive/MyDrive/CURIS/VeriX/networks/'

In [19]:
SEED=137

In [20]:
# Read and process dataset
import pickle
with open(input_path, 'rb') as handle:
    gtsrb = pickle.load(handle)

x_train, y_train, x_valid, y_valid, x_test, y_test = gtsrb.values()
x_train = x_train/255
x_valid = x_valid/255
x_test = x_test/255

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)

In [21]:
def eval_and_save(model):
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  model.summary()
  model.save(output_path + model.name + '.h5')
  model_proto, _ = tf2onnx.convert.from_keras(model, output_path=output_path + model.name + '.onnx')

10x2 fully connected

In [22]:
model = Sequential(name='gtsrb-10x2')
model.add(Flatten(name='input'))
model.add(Dense(10, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 11s 17ms/step - loss: 1.8579 - accuracy: 0.3912 - val_loss: 1.7076 - val_accuracy: 0.5193
Epoch 2/20
138/138 [==============================] - 2s 12ms/step - loss: 1.2730 - accuracy: 0.6161 - val_loss: 1.2777 - val_accuracy: 0.5966
Epoch 3/20
138/138 [==============================] - 2s 11ms/step - loss: 0.9954 - accuracy: 0.7081 - val_loss: 1.1029 - val_accuracy: 0.6556
Epoch 4/20
138/138 [==============================] - 2s 15ms/step - loss: 0.8561 - accuracy: 0.7459 - val_loss: 0.9839 - val_accuracy: 0.6966
Epoch 5/20
138/138 [==============================] - 1s 10ms/step - loss: 0.7609 - accuracy: 0.7830 - val_loss: 0.8977 - val_accuracy: 0.7227
Epoch 6/20
138/138 [==============================] - 1s 11ms/step - loss: 0.6890 - accuracy: 0.8091 - val_loss: 0.9424 - val_accuracy: 0.7155
Epoch 7/20
138/138 [==============================] - 1s 7ms/step - loss: 0.6415 - accuracy: 0.8214 - val_loss: 0.7920 - val_accuracy: 0.7691

In [23]:
eval_and_save(model)

Test loss: 0.5362910628318787
Test accuracy: 0.8476851582527161
Model: "gtsrb-10x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                30730     
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 logit (Dense)               (None, 10)                110       
                                                                 
Total params: 30,950
Trainable params: 30,950
Non-trainable params: 0
_________________________________________________________________


30x2 fully connected

In [24]:
model = Sequential(name='gtsrb-30x2')
model.add(Flatten(name='input'))
model.add(Dense(30, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(30, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 2s 6ms/step - loss: 1.6794 - accuracy: 0.4917 - val_loss: 1.2460 - val_accuracy: 0.6580
Epoch 2/20
138/138 [==============================] - 1s 4ms/step - loss: 0.9599 - accuracy: 0.7467 - val_loss: 0.8592 - val_accuracy: 0.7594
Epoch 3/20
138/138 [==============================] - 1s 4ms/step - loss: 0.6553 - accuracy: 0.8296 - val_loss: 0.7457 - val_accuracy: 0.7807
Epoch 4/20
138/138 [==============================] - 1s 4ms/step - loss: 0.5132 - accuracy: 0.8670 - val_loss: 0.6615 - val_accuracy: 0.8116
Epoch 5/20
138/138 [==============================] - 1s 5ms/step - loss: 0.4232 - accuracy: 0.8974 - val_loss: 0.4753 - val_accuracy: 0.8758
Epoch 6/20
138/138 [==============================] - 1s 4ms/step - loss: 0.3637 - accuracy: 0.9078 - val_loss: 0.5401 - val_accuracy: 0.8362
Epoch 7/20
138/138 [==============================] - 1s 5ms/step - loss: 0.3273 - accuracy: 0.9187 - val_loss: 0.6577 - val_accuracy: 0.8019
Epoch 

In [25]:
eval_and_save(model)

Test loss: 0.4738653600215912
Test accuracy: 0.8757715821266174
Model: "gtsrb-30x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 30)                92190     
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 logit (Dense)               (None, 10)                310       
                                                                 
Total params: 93,430
Trainable params: 93,430
Non-trainable params: 0
_________________________________________________________________


100x2 fully connected

In [26]:
model = Sequential(name='gtsrb-100x2')
model.add(Flatten(name='input'))
model.add(Dense(100, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(100, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 2s 6ms/step - loss: 1.4494 - accuracy: 0.5556 - val_loss: 1.0476 - val_accuracy: 0.6913
Epoch 2/20
138/138 [==============================] - 1s 4ms/step - loss: 0.7313 - accuracy: 0.7929 - val_loss: 0.7236 - val_accuracy: 0.7928
Epoch 3/20
138/138 [==============================] - 1s 4ms/step - loss: 0.4884 - accuracy: 0.8696 - val_loss: 0.4966 - val_accuracy: 0.8845
Epoch 4/20
138/138 [==============================] - 1s 4ms/step - loss: 0.3808 - accuracy: 0.8962 - val_loss: 0.8183 - val_accuracy: 0.7667
Epoch 5/20
138/138 [==============================] - 1s 4ms/step - loss: 0.3400 - accuracy: 0.9064 - val_loss: 0.4750 - val_accuracy: 0.8464
Epoch 6/20
138/138 [==============================] - 1s 6ms/step - loss: 0.2436 - accuracy: 0.9385 - val_loss: 0.4344 - val_accuracy: 0.8797
Epoch 7/20
138/138 [==============================] - 1s 7ms/step - loss: 0.2566 - accuracy: 0.9285 - val_loss: 0.3740 - val_accuracy: 0.8913
Epoch 

In [27]:
eval_and_save(model)

Test loss: 0.38844770193099976
Test accuracy: 0.917438268661499
Model: "gtsrb-100x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 100)               307300    
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 logit (Dense)               (None, 10)                1010      
                                                                 
Total params: 318,410
Trainable params: 318,410
Non-trainable params: 0
_________________________________________________________________


simple cnn model

In [28]:
x_test.shape

(6480, 32, 32, 3)

In [29]:
model = Sequential(name='gtsrb-cnn-3x3')
model.add(Conv2D(4, (3, 3), name='conv_1', input_shape=(32, 32, 3), kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(4, (3, 3), name='conv_2', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Flatten())
model.add(Dense(20, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 9s 10ms/step - loss: 1.3080 - accuracy: 0.5827 - val_loss: 0.7338 - val_accuracy: 0.8005
Epoch 2/20
138/138 [==============================] - 1s 5ms/step - loss: 0.3859 - accuracy: 0.8921 - val_loss: 0.3915 - val_accuracy: 0.9039
Epoch 3/20
138/138 [==============================] - 1s 5ms/step - loss: 0.2306 - accuracy: 0.9395 - val_loss: 0.3541 - val_accuracy: 0.9174
Epoch 4/20
138/138 [==============================] - 1s 5ms/step - loss: 0.1695 - accuracy: 0.9552 - val_loss: 0.3113 - val_accuracy: 0.9251
Epoch 5/20
138/138 [==============================] - 1s 5ms/step - loss: 0.1366 - accuracy: 0.9642 - val_loss: 0.2727 - val_accuracy: 0.9391
Epoch 6/20
138/138 [==============================] - 1s 5ms/step - loss: 0.1158 - accuracy: 0.9679 - val_loss: 0.3780 - val_accuracy: 0.9150
Epoch 7/20
138/138 [==============================] - 1s 6ms/step - loss: 0.0992 - accuracy: 0.9732 - val_loss: 0.3130 - val_accuracy: 0.9295
Epoch

In [30]:
eval_and_save(model)

Test loss: 0.510371744632721
Test accuracy: 0.927314817905426
Model: "gtsrb-cnn-3x3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 30, 30, 4)         112       
                                                                 
 conv_2 (Conv2D)             (None, 28, 28, 4)         148       
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 20)                62740     
                                                                 
 logit (Dense)               (None, 10)                210       
                                                                 
Total params: 63,210
Trainable params: 63,210
Non-trainable params: 0
_____________________________________________________

cnn with max pooling and dropout

In [31]:
model = Sequential(name='gtsrb-sota')
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dropout(0.5, seed=SEED))
model.add(Dense(200, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))
# model.add(Activation('softmax', name='output'))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_valid, y_valid))

Epoch 1/20
138/138 [==============================] - 7s 15ms/step - loss: 1.0477 - accuracy: 0.6212 - val_loss: 0.2680 - val_accuracy: 0.9275
Epoch 2/20
138/138 [==============================] - 1s 10ms/step - loss: 0.1835 - accuracy: 0.9393 - val_loss: 0.0780 - val_accuracy: 0.9826
Epoch 3/20
138/138 [==============================] - 1s 10ms/step - loss: 0.0860 - accuracy: 0.9741 - val_loss: 0.0619 - val_accuracy: 0.9836
Epoch 4/20
138/138 [==============================] - 1s 10ms/step - loss: 0.0522 - accuracy: 0.9829 - val_loss: 0.0410 - val_accuracy: 0.9879
Epoch 5/20
138/138 [==============================] - 1s 10ms/step - loss: 0.0458 - accuracy: 0.9852 - val_loss: 0.0451 - val_accuracy: 0.9874
Epoch 6/20
138/138 [==============================] - 1s 10ms/step - loss: 0.0215 - accuracy: 0.9942 - val_loss: 0.0365 - val_accuracy: 0.9903
Epoch 7/20
138/138 [==============================] - 1s 10ms/step - loss: 0.0209 - accuracy: 0.9940 - val_loss: 0.0408 - val_accuracy: 0.9870

In [32]:
eval_and_save(model)

Test loss: 0.08789122104644775
Test accuracy: 0.9867283701896667
Model: "gtsrb-sota"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0 